In [ ]:
# mount Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# pip installs
!pip install -q -U datasets transformers

In [ ]:
# load original dataset
from datasets import load_dataset
eval_dataset = load_dataset("FreedomIntelligence/medical-o1-verifiable-problem", split="train[:1%]")

README.md: 0.00B [00:00, ?B/s]

medical_o1_verifiable_problem.json:   0%|          | 0.00/12.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/40644 [00:00<?, ? examples/s]

In [ ]:
# load model responses
fn_out = "/content/drive/MyDrive/266 Final Project/model_output/lora-sft-r32-output-reformatted.txt"
f = open(fn_out, 'r')
L = f.readlines()
f.close()
L = L[1:]
len(L)

510

In [ ]:
# load scores
import pickle
fn_pkl = fn_out.replace('.txt','.pkl')
with open(fn_pkl, 'rb') as f_pkl:
  scores = pickle.load(f_pkl)

In [ ]:
import numpy as np
scr = 0.9*scores['entF'] + 0.1*scores['slf']
use_me = np.argmax(scr,axis=1)
use_me

array([1, 4, 1, 3, 1, 2, 3, 1, 0, 0, 2, 3, 0, 3, 0, 0, 4, 0, 1, 2, 2, 0,
       4, 2, 2, 2, 0, 2, 0, 1, 3, 1, 0, 4, 4, 2, 3, 0, 2, 4, 4, 1, 3, 1,
       0, 2, 3, 3, 1, 4, 3, 2, 1, 2, 1, 4, 1, 3, 3, 3, 4, 3, 3, 4, 4, 3,
       1, 2, 4, 4, 3, 1, 3, 3, 3, 0, 4, 2, 0, 2, 2, 4, 1, 2, 0, 4, 4, 3,
       1, 3, 4, 4, 3, 1, 2, 4, 0, 2, 2, 4, 4, 0])

In [ ]:
out = []
for idx_sample,idx_ex in enumerate(use_me):

  txt = L[5*idx_sample + idx_ex]
  assert idx_sample == int(txt.split('*****!!!!!')[0])
  out.append(
      {"Question": eval_dataset['Open-ended Verifiable Question'][idx_sample],
       "Answer": txt.split('*****!!!!!')[1]}
  )


In [ ]:
import pandas as pd
df = pd.DataFrame(out)
df

,Question,Answer
0,An 88-year-old woman with osteoarthritis is ex...,The most likely cause of the gastrointestinal ...
1,In the context of disseminated intravascular c...,In the context of disseminated intravascular c...
2,"In a 3-year-old boy with severe diarrhea, vomi...","In a 3-year-old boy with severe diarrhea, vomi..."
3,Based on the chest radiograph and abdominal CT...,"Based on the information provided, the probabl..."
4,What is one potential side effect that is not ...,One potential side effect that is not associat...
...,...,...
97,A 10-year-old boy presents with recurring epis...,The most likely underlying genetic cause of th...
98,A 40-year-old woman with a history of breast c...,The primary mechanism of action of the analges...
99,A 35-year-old female presents with a solitary ...,The appropriate surgical management for a 35-y...
100,A 41-year-old woman presents with an 8-hour hi...,The clinical findings and laboratory results s...


In [42]:
from datasets import Dataset
dataset = Dataset.from_pandas(df)

In [49]:
# huggingface login
from google.colab import userdata
from huggingface_hub import notebook_login

notebook_login()

In [52]:
dataset.push_to_hub("erica-landreth/bootstrap-test",token=userdata.get('notebook2'))

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading...:   0%|          | 0.00/46.2k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/erica-landreth/bootstrap-test/commit/2f757b9d5c95ecbf4a586b27e82c5106982fafeb', commit_message='Upload dataset', commit_description='', oid='2f757b9d5c95ecbf4a586b27e82c5106982fafeb', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/erica-landreth/bootstrap-test', endpoint='https://huggingface.co', repo_type='dataset', repo_id='erica-landreth/bootstrap-test'), pr_revision=None, pr_num=None)